In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
from multiprocess import Pool

from scipy.spatial.kdtree import KDTree
from scipy.spatial.distance import cdist, pdist

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

In [3]:
n_radial = 4
# radial_bins = np.logspace(0, 2, 10, base=10)
radial_bins = np.linspace(0, 100, n_radial+1)

n_angular = 8
angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

n_orientation_bins = 10
orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

size_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

n_edge_length_bins = 10
edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]

n_edge_direction_bins = 10
edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)

In [24]:
neighborhood_radius_um = 50
neighborhood_radius = neighborhood_radius_um / XY_PIXEL_DISTANCE_LOSSLESS
print 'neighborhood_radius = %.2f' % neighborhood_radius

neighborhood_radius = 108.70


In [39]:
for stack in ['MD594']:
    
    first_section, last_section = metadata_cache['section_limits'][stack]
    
#     for sec in range(first_section, last_section+1):
    for sec in [203]:
        
        cell_centroids = load_cell_data('centroid', stack, sec)

        cell_orientations = load_cell_data('orientation', stack, sec)
        cell_orientations = np.array(map(normalize_angle, cell_orientations))
        
        large_cell_indices = load_cell_data('largeCellIndices', stack=stack, sec=sec, ext='bp')
        
        t = time.time()

        tree = KDTree(cell_centroids)
        
        def get_neighbors(cell_ind):
            """
            Return neighbor cell indices sorted by distance.
            """
            neighbor_cell_indices = np.array(list(set(tree.query_ball_point(cell_centroids[i], r=neighborhood_radius)) - {i}))
            if len(neighbor_cell_indices) == 0:
                return None
            ngbr_vecs = cell_centroids[cell_ind] - cell_centroids[neighbor_cell_indices]
            neighbor_distances = np.sqrt(np.sum(ngbr_vecs**2, axis=1))
            order = np.argsort(neighbor_distances)
            
            radial_indices, angular_indices = \
            allocate_radial_angular_bins(ngbr_vecs, cell_orientations[cell_ind],
                                         angular_bins=angular_bins, radial_bins=radial_bins)
            
            return neighbor_cell_indices[order], ngbr_vecs[order], radial_indices[order], angular_indices[order]
                
        pool = Pool(12)
        neighbor_info = pool.map(get_neighbors, large_cell_indices)
        pool.terminate()
        pool.join()

        sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

        ##############################

#         neighbor_info = {'neighbors': neighbors_sorted_by_distance,
#          'neighbor_vectors': neighbor_vectors_sorted_by_distance,
#          'radial_indices': radial_indices_all_sorted_by_distance,
#          'angular_indices': angular_indices_all_sorted_by_distance
#         }

#         fp = get_cell_data_filepath(what='neighbor_info', stack=stack, sec=sec, ext='pkl')
#         create_parent_dir_if_not_exists(fp)
#         save_pickle(neighbor_info, fp)

Neighbor search: 7.59 seconds


In [40]:
large_cell_indices

array([    1,     2,     3, ..., 90680, 90683, 90687])

In [41]:
neighbor_info[0]